# Singapore Flat Resale Prices 1990-Present

- Month - Month of sale
- Type - Designated residential area with its own amenities, infrastructure, and community facilities
- Flat Type - Classification of units by room size. They range from 2 to 5 rooms, 3Gen units, and Executive units.
- Block - A HDB building comprising multiple flats or apartments
- Street Name - Name of the road the HDB flat is located along
- Storey Range - Estimated range of floors the unit sold was located on
- Floor Area - Total interior space within the unit, measured in square meters
- Flat Model - Classification of units by generation of which the flat was made, ranging from New Generation, DBSS, Improved, Apartment
- Lease Commence Date - Starting point of a lease agreement, marking the beginning of the lease term during which the tenant has the right to use and occupy the leased property
- Resale Price - Cost of the flat sold

In [2]:
from src.config import RAW_DATA_DIR, EXTERNAL_DATA_DIR

2025-07-14 14:38:30.013 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /Users/Edward/Documents/GitHub/sg-flat-resale


In [3]:
import numpy as np
import pandas as pd

In [4]:
raw_data_files = ['ResaleFlatPricesBasedonApprovalDate19901999.csv',
                  'ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv',
                  'ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv',
                  'ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv',
                  'ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv']

dfs = [pd.read_csv(RAW_DATA_DIR / file) for file in raw_data_files]
df = pd.concat(dfs, ignore_index=True)
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
941100,2024-11,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,720000.0,91 years 10 months
941101,2024-11,YISHUN,5 ROOM,850,YISHUN ST 81,01 TO 03,122.0,Improved,1988,670000.0,62 years 09 months
941102,2024-11,YISHUN,EXECUTIVE,405,YISHUN AVE 6,04 TO 06,148.0,Maisonette,1988,855500.0,62 years 10 months
941103,2024-11,YISHUN,EXECUTIVE,356,YISHUN RING RD,01 TO 03,146.0,Maisonette,1988,930000.0,62 years 10 months


In [5]:
# Combine "MULTI-GENERATION" and "MULTI GENERATION" flat types
df['flat_type'] = df['flat_type'].replace({'MULTI GENERATION' : 'MULTI-GENERATION'})

# Create date column by setting the day from each month to 01 since the original time format provided is has no day provided
df['date'] = pd.to_datetime(df['month'], format='%Y-%m')

# Create year column
df['year'] = df['date'].dt.strftime('%Y').astype('int64')

# Rename lease_commence_date and add year_leased column
df = df.rename({'lease_commence_date':'lease_year'}, axis=1)
df['years_leased'] = df['year'] - df['lease_year']

# Changing column entries to proper case
df['town'] = df['town'].apply(lambda x: x.title())
df['street_name'] = df['street_name'].apply(lambda x: x.title())
df['flat_model'] = df['flat_model'].apply(lambda x: x.title())
df['flat_type'] = df['flat_type'].apply(lambda x: x.title())

# Split storey_range into 2 columns and calculate number of storeys
df[['start_floor', 'end']] = df['storey_range'].str.extract(r'(\d+)\s+TO\s+(\d+)').astype(int)
df['storey_count'] = df['end'] - df['start_floor']

# Rearrange columns
df = df.loc[:, ['date', 'year', 'town', 'street_name', 'block', 'flat_type', 'flat_model', 'storey_count', 
                'start_floor', 'floor_area_sqm', 'lease_year', 'years_leased', 'resale_price']]

In [6]:
df.head()

,date,year,town,street_name,block,flat_type,flat_model,storey_count,start_floor,floor_area_sqm,lease_year,years_leased,resale_price
0,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,9000.0
1,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,4,31.0,1977,13,6000.0
2,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,8000.0
3,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,7,31.0,1977,13,6000.0
4,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,216,3 Room,New Generation,2,4,73.0,1976,14,47200.0


The data source does not indicate whether the resale prices are adjusted for inflation. It may be necessary to adjust the prices for inflation.

In [20]:
infl = pd.read_csv(EXTERNAL_DATA_DIR / 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_37769.csv')
infl

ParserError: Error tokenizing data. C error: Expected 3 fields in line 5, saw 70


In [15]:
# Create dataframe for percent inflation figures in Singapore
infl = pd.read_csv(EXTERNAL_DATA_DIR / 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_37769.csv')
infl = infl.loc[infl['Country Name'] == 'Singapore', '1990':]
infl = infl.rename({208:'infl'})

# Transpose inflation dataframe
infl = infl.T
infl

ParserError: Error tokenizing data. C error: Expected 3 fields in line 5, saw 70


In [8]:
# Create dataframe for percent inflation figures in Singapore
infl = pd.read_csv(EXTERNAL_DATA_DIR / 'API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_77.csv')
infl = infl.loc[infl['Country Name'] == 'Singapore', '1990':]
infl = infl.rename({208:'infl'})

# Add 2024 inflation as 0
infl['2024'] = 0

# Transpose inflation dataframe
infl = infl.T

# Calculate cumulative inflation figures (2024)
infl['cum_infl'] = infl.iloc[::-1,:].cumsum()

# Reset index, rename index column to 'year', set the dtype to int64
infl = infl.reset_index()
infl = infl.rename({'index':'year'}, axis=1)
infl['year'] = infl['year'].astype('int64')

In [9]:
# Merge original dataframe with the inflation dataframe on year
df = pd.merge(df, infl[['year', 'cum_infl']], on='year', how='left')

# Add column for resale price adjusted by inflation
df['infl_adj_price'] = df['resale_price'] + df['resale_price']*df['cum_infl']/100
df['infl_adj_price'] = df['infl_adj_price'].round(1)
df = df.drop('cum_infl', axis=1)
df.head()

,date,year,town,street_name,block,flat_type,flat_model,storey_count,start_floor,floor_area_sqm,lease_year,years_leased,resale_price,infl_adj_price
0,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,9000.0,14748.5
1,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,4,31.0,1977,13,6000.0,9832.3
2,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,10,31.0,1977,13,8000.0,13109.8
3,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,309,1 Room,Improved,2,7,31.0,1977,13,6000.0,9832.3
4,1990-01-01,1990,Ang Mo Kio,Ang Mo Kio Ave 1,216,3 Room,New Generation,2,4,73.0,1976,14,47200.0,77347.7


There is now a column for the resale price of each unit adjusted for inflation (2024)  
Source of inflation figures: World Bank (https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG?contextual=default&end=2023&locations=SG&start=1961&view=chart)

In [10]:
df.describe()

,date,year,storey_count,start_floor,floor_area_sqm,lease_year,years_leased,resale_price,infl_adj_price
count,941105,941105.000000,941105.000000,941105.000000,941105.000000,941105.000000,941105.000000,9.411050e+05,9.411050e+05
mean,2006-11-23 23:34:11.583617024,2006.432752,2.014532,6.697608,95.686705,1988.422273,18.010479,3.252519e+05,4.151768e+05
min,1990-01-01 00:00:00,1990.000000,2.000000,1.000000,28.000000,1966.000000,-2.000000,5.000000e+03,8.193600e+03
25%,1999-02-01 00:00:00,1999.000000,2.000000,4.000000,73.000000,1981.000000,9.000000,1.950000e+05,2.773563e+05
50%,2005-07-01 00:00:00,2005.000000,2.000000,7.000000,93.000000,1986.000000,16.000000,3.000000e+05,3.950378e+05
75%,2014-10-01 00:00:00,2014.000000,2.000000,10.000000,113.000000,1996.000000,25.000000,4.220000e+05,5.276744e+05
max,2024-11-01 00:00:00,2024.000000,4.000000,49.000000,366.700000,2021.000000,58.000000,1.588000e+06,1.588000e+06
std,NaN,9.498936,0.169860,4.841988,25.801141,10.809251,10.836415,1.746709e+05,1.908177e+05


In [11]:
df[df['years_leased']<0]

,date,year,town,street_name,block,flat_type,flat_model,storey_count,start_floor,floor_area_sqm,lease_year,years_leased,resale_price,infl_adj_price
23271,1991-11-01,1991,Bishan,Bishan St 23,227,4 Room,Model A,2,4,103.0,1992,-1,135000.0,216555.4
34246,1992-08-01,1992,Tampines,Tampines St 44,476,5 Room,Model A,2,4,132.0,1993,-1,200000.0,313971.4
46535,1993-06-01,1993,Woodlands,Woodlands St 32,321,5 Room,Improved,2,1,127.0,1994,-1,145000.0,224347.8
69033,1994-05-01,1994,Choa Chu Kang,Choa Chu Kang St 51,528,Executive,Apartment,2,7,153.0,1995,-1,350000.0,533516.7
69609,1994-05-01,1994,Pasir Ris,Pasir Ris Dr 3,631,4 Room,Model A,2,10,104.0,1995,-1,160000.0,243893.3
69977,1994-05-01,1994,Tampines,Tampines St 32,332,Executive,Apartment,2,4,147.0,1996,-2,356000.0,542662.7
71709,1994-06-01,1994,Pasir Ris,Pasir Ris St 11,143,4 Room,Model A,2,4,105.0,1995,-1,180000.0,274380.0
76424,1994-08-01,1994,Woodlands,Marsiling Rd,136,Executive,Apartment,2,1,173.0,1995,-1,330000.0,503030.0
77301,1994-09-01,1994,Choa Chu Kang,Choa Chu Kang St 51,503,5 Room,Improved,2,13,127.0,1995,-1,235000.0,358218.4
77902,1994-09-01,1994,Queenstown,Mei Ling St,150,Executive,Apartment,2,10,142.0,1995,-1,532000.0,810945.4
